This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [93]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

#print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [94]:
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [95]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
req = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2020-12-01&api_key={}'.format(API_KEY))

In [96]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(req.json())

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2020-12-01', 'end_date': '2020-12-01', 'frequency': 'daily', 'data': [['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [97]:
# 1. Collect the data
req = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-02&end_date=2017-12-29&api_key={}'.format(API_KEY))
response = req.json()
#response

In [98]:
# 2. Convert to Python dictionary
# --> key = Date, values = [Open, High, Low, Close, Change, Traded Volume, Turnover,
# Last Price of the Day, Daily Traded Units, Daily Turnover]
data = req.json()['dataset_data']['data']
data_dict = {day[0]:day[1:] for day in data}
#data_dict

In [99]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period
sorted_open = sorted(data_dict.items(), key = lambda i: i[0][1], reverse = True)
highest = sorted_open[0][1][0]
lowest = sorted_open[len(sorted_open)-1][1][0]
print("Highest opening price: {}\nLowest opening price: {}".format(highest, lowest))

Highest opening price: 51.76
Lowest opening price: 34.99


In [100]:
# 4. What was the largest change in any one day (based on High and Low price)?
changes = [float(day[1][1] - day[1][2]) for day in data_dict.items()]
print("Largest change in a day: {:.2f}".format(max(changes)))

Largest change in a day: 2.81


In [101]:
# 5. What was the largest change between any two days (based on Closing Price)?
def changes(gdict):
    values = list(gdict.values())
    largest_change = 0
    for count, day in enumerate(values[:-1]):
        largest_change = max(largest_change, abs(day[3] - values[count+1][3]))
    return largest_change

change2 = changes(data_dict)
print("Largest change between any two days: {:.2f}".format(change2))

Largest change between any two days: 2.56


In [102]:
# 6. What was the average daily trading volume during this year?
traded_volume = [day[6] for day in data_dict.values()]
avg_traded_volume = sum(traded_volume)/len(traded_volume)
print("Average daily trading volume: {:.2f}".format(avg_traded_volume))

Average daily trading volume: 3853589.45


In [103]:
# 7. What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
import statistics

median_traded_volume = statistics.median(traded_volume)
print("Median daily trading volume: {:.2f}".format(median_traded_volume))

Median daily trading volume: 3292223.00
